In [3]:
import heapq

In [4]:
class Node:
    
    def __init__(self,name):
        self._name = name
        self._neighbors = {}
    
    def __str__(self):
        msg = "Node: {}\nEdges: {}\n".format(self._name,len(self))
        msg += ''.join("  Edge: ({},{}) Weight: {}\n".format(self._name,n,w) for (n,w) in self._neighbors.items())
        return msg
    
    def __len__(self):
        return len(self._neighbors)
    
    def __contains__(self,item):
        if isinstance(item, Node):
            item = item._name
        return item in self._neighbors
    
    def __getitem__(self, key):
        if isinstance(key, Node):
            key = key._name
        return self._neighbors.get(key, None)
    
    def __setitem__(self, key, item):
        if isinstance(key,Node):
            key = key._name
        if self._name != key:
            self._neighbors[key] = max(self._neighbors.get(key,item),item)
    
    def __eq__(self, other):
        return self._name == other._name
    
    def __ne__(self, other):
        return self._name != other._name
     
    def is_neighbor(self, name):
        return name in self
    
    def update(self, name, weight):
        self[name] = weight
        
    def update(self, node):
        if self == other:
            for neighbor, weight in other._neighbors.items():
                self[neighbor] = weight
            
    def remove_neighbor(self, name):
        if name in self:
            del self._neighbors[name]
    
    def is_isolated(self):
        return not self._neighbors
    
    def name(self):
        return self._name
    
    def edges(self):
        return self._neighbors.items()
    



In [55]:
class Graph(object):
    
    def __init__(self, name, nodes=[]):
        self._name = name
        self._nodes = {}
        for node in nodes:
            self.update(node)
    
    def __str__(self):
        msg="Graph: {}\nNodes: {}\n".format(self._name, len(self))
        msg += ''.join("  Name: {} Neighbors: {}\n".format(node,len(list(self.neighbors(node)))) for node in self._nodes)
        edges = ''.join("  Edge: ({},{}) Weight: {}\n".format(frm_n,to_n,w) for frm_n in self._nodes \
                        for (to_n,w) in self.edges_from(frm_n))
        msg += "Edges: {}\n".format(edges.count("Edge: "))
        msg += edges
        return msg
    
    def __len__(self):
        return len(self._nodes)
    
    def __contains__(self,key):
        if isinstance(key,Node):
            key = key._name
        return key in self._nodes

    def __getitem__(self,name):
        return self._nodes[name]
    
    def __add__(self,other):
        new_graph = Graph(self._name + "_" + other._name,[])
        new_graph._nodes = self._nodes.copy()
        for node in other._nodes.values():
            new_graph.update(node)
        return new_graph
    
    def update(self,node):
        if node in self:
            self._nodes[node.name()].update(node)
        else:
            self._nodes[node.name()] = node
        
    def edges_from(self, frm_name):
        return ((to_name,weight) for (to_name, weight) in self[frm_name].edges() if to_name in self)
    
    def neighbors(self, of_name):
        return(node for node in self._nodes if node in self._nodes[of_name])
        
    def remove_node(self, name):
        if name in self:
            del self._nodes[name]
    
    def is_edge(self, frm_name, to_name):
        return frm_name in self and to_name in self and to_name in self[frm_name]
    
    def add_edge(self, frm_name, to_name, weigth):
        if frm_name in self and to_name in self:
            self._nodes[frm_name][to_name] = weigth
    
    def remove_edge(self,frm_name, to_name):
        if frm_name in self and to_name in self:
            self[frm_name].remove_neighbor(to_name)
    
    def get_edge_weight(self,frm_name, to_name):
        return_value = None
        if frm_name in self and to_name in self:
            return_value = self[frm_name][to_name]
        return return_value
    
    def get_path_weight(self, path):
        #what should be the return value of a path of one node in the graph? (currnet 0)
        path_weight = None
        if path:
            to_nodes = iter(path)
            next(to_nodes)
            edge_weights = [self.get_edge_weight(frm_node,to_node) for frm_node,to_node in zip(path,to_nodes)]
            if None not in edge_weights:
                path_weight = sum(edge_weights)
        return path_weight
  
    def is_reachable(self,frm_name, to_name):
        #what should be the return value of a noe to its self? (current True)
        recheable = False
        if frm_name in self and to_name in self:
            visited = set()
            seen = {frm_name}
            while seen and to_name not in seen:
                visiting = seen.pop()
                seen.update(node for node in self.neighbors(visiting) if node not in visited)
                visited.add(visiting)
            recheable = to_name in seen
        return recheable
                
    
    def find_shortest_path(self, frm_name, to_name):
        #Dijkstra
        shortest_path = None
        if frm_name in self and to_name in self:
            pq = []
            predecessor = {}
            distance = {frm_name:0}
            visited = set()
            heapq.heappush(pq,(0,frm_name))
            while pq and to_name not in visited:
                visiting_distance, visiting_node = heapq.heappop(pq)
                if visiting_node not in visited:
                    for peer, weight in self.edges_from(visiting_node):
                        if peer not in distance or distance[peer] > visiting_distance + weight:
                            #Relaxation
                            distance[peer] = visiting_distance + weight
                            predecessor[peer] = visiting_node
                            heapq.heappush(pq,(distance[peer],peer))
                    visited.add(visiting_node)
            if to_name in visited:
                #Creating the path
                shortest_path = [to_name]
                next_hop = to_name
                while next_hop != frm_name:
                    next_hop = predecessor[next_hop]
                    shortest_path.append(next_hop)
                shortest_path.reverse()
        return shortest_path
            
        
     

In [56]:
class NonDirectionalGraph(Graph):
    
    def update(self, node):
        super().update(node)
        node = self[node.name()]
        for peer in self._nodes.values():
            if peer in node:
                peer[node] = node[peer]
                #node[peer] = peer[node]
            if node in peer:
                node[peer] = peer[node]
                
    def add_edge(self, frm_name, to_name, weigth):
        if frm_name in self and to_name in self:
            self._nodes[frm_name][to_name] = weigth
            slef._nodes[to_name][frm_name] = weigth
    
    def remove_edge(self,frm_name, to_name):
        if frm_name in self and to_name in self:
            self[frm_name].remove_neighbor(to_name)
            self[to_name].remove_neighbor(frm_name)
            
    def __add__(self, other):
        new_graph = super().__add__(other)
        new_non_directional_graph = NonDirectionalGraph(new_graph._name,[])
        new_non_directional_graph._nodes = new_graph._nodes
        return new_non_directional_graph
    